In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import streamlit as st
import sqlite3
from dotenv import load_dotenv
from unidecode import unidecode
from itertools import repeat

load_dotenv()

db_connection = sqlite3.connect(f"{os.environ.get('DB_PATH')}")
db_cursor = db_connection.cursor()


def create_sheets_link():
    sheet_id = os.environ.get("SHEET_ID")
    page_name = os.environ.get("PAGE_NAME")
    complete_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={page_name}"
    return complete_url


def retrieve_sheets_data() -> pd.DataFrame:
    df = pd.read_csv(create_sheets_link())
    df[df.select_dtypes(include="float").columns] = df[
        df.select_dtypes(include="float").columns
    ].fillna(0)
    df.columns = [
        unidecode(col.lower().replace(" ", "_").replace("\n", ""))
        for col in df.columns.tolist()
    ]
    df = df.drop("volume_total", axis=1)
    df["series_personalizadas"] = df["series_personalizadas"].fillna("0")
    df["series_personalizadas_calculadas"] = (
        df["series_personalizadas"]
        .apply(lambda x: eval(str(x).replace("x", "*")))
        .apply(lambda x: sum(x) if type(x) != int else x)
    )
    df["volume_total_kg"] = (df["series"] * df["repeticoes"] * df["peso"]) + df[
        "series_personalizadas_calculadas"
    ]
    df["dia"] = df["dia"].fillna(method="ffill")
    return df


def save_df_to_db(df: pd.DataFrame, db_connection) -> None:
    df.to_sql("gym_data", db_connection, if_exists="replace", index=False)


df = retrieve_sheets_data()
save_df_to_db(df=df, db_connection=db_connection)

In [ ]:
drop_table_string = f"""DROP TABLE IF EXISTS gym_data"""
create_table_string = f"""
CREATE TABLE IF NOT EXISTS gym_data{tuple(df.columns)}"""
values_binding = tuple(repeat("?", len(list(df.columns))))
data = list(df.to_records(index=False))
db_cursor.execute(drop_table_string)
db_connection.commit()
db_cursor.execute(create_table_string)
db_connection.commit()
for row in data:
    db_cursor.execute(f"INSERT INTO gym_data VALUES {row}")
# db_cursor.executemany(f"INSERT INTO gym_data VALUES ('?', '?', '?', '?', '?', '?', '?', '?', '?', '?') ;",(data,))
db_connection.commit()

In [6]:
db_cursor.execute("PRAGMA table_info(gym_data)")
db_connection.commit()
db_cursor.fetchall()

[(0, 'dia', 'TEXT', 0, None, 0),
 (1, 'grupo_geral', 'TEXT', 0, None, 0),
 (2, 'grupo_secundario', 'TEXT', 0, None, 0),
 (3, 'exercicio', 'TEXT', 0, None, 0),
 (4, 'series', 'REAL', 0, None, 0),
 (5, 'repeticoes', 'REAL', 0, None, 0),
 (6, 'peso', 'REAL', 0, None, 0),
 (7, 'series_personalizadas', 'TEXT', 0, None, 0),
 (8, 'series_personalizadas_calculadas', 'INTEGER', 0, None, 0),
 (9, 'volume_total_kg', 'REAL', 0, None, 0)]

Peso médio total por grupamento geral:

In [8]:
(
    df.groupby("grupo_geral")["volume_total_kg"].sum()
    / df.groupby("grupo_geral").size()
).round(2)

grupo_geral
A    2276.24
B    2808.88
C    1122.86
dtype: float64

Volume médio por grupamento secundario:

In [9]:
(
    df.groupby("grupo_secundario")["volume_total_kg"].sum()
    / df.groupby("grupo_secundario").size()
).round(2)

grupo_secundario
Antebraços    1400.00
Costas        2182.62
Ombros        1015.00
Peito         2359.44
Pernas        2808.88
Triceps       1200.00
dtype: float64

In [12]:
gtx_1660 = [134, 38, 88, 77, 88, 36, 103, 44, 63, 62, 38, 60]
rtx_2660 = [182, 57, 111, 93, 127, 49, 138, 62, 87, 70, 48, 69]
mean_fps_gtx_1660 = round(sum(gtx_1660) / len(gtx_1660), 2)
mean_fps_rtx_2060 = round(sum(rtx_2660) / len(rtx_2660), 2)
print(
    f"""FPS mínimo médio GTX 1660 Super 6gb: {mean_fps_gtx_1660}
        \n FPS mínimo médio RTX 2060 Super 8gb: {mean_fps_rtx_2060}"""
)
performance_variation = round(((mean_fps_rtx_2060 / mean_fps_gtx_1660) - 1) * 100, 2)
print(
    f"Variação percentual de performance RTX 2060S vs GTX 1660S: {performance_variation}%"
)
gtx_1600_price = 715.29
rtx_2060_price = 1123.10
price_variation = round((rtx_2060_price / gtx_1600_price - 1) * 100, 2)
print(f"Variação percentual de preço RTX 2060S vs GTX 1660S: {price_variation}%")

FPS mínimo médio GTX 1660 Super 6gb: 69.25
        
 FPS mínimo médio RTX 2060 Super 8gb: 91.08
Variação percentual de performance RTX 2060S vs GTX 1660S: 31.52%
Variação percentual de preço RTX 2060S vs GTX 1660S: 57.01%
